# Etapa 01 - Pré-processamento de Dados
## Classificação de Desinformação Digital - Liga Acadêmica de IA (Ligia/UFPE) 2026

**Objetivo:** Preparar os dados brutos para a etapa de modelagem, executando as seguintes transformações em ordem determinística: remoção de data leakage, extração de features de estilo, lematização e vetorização TF-IDF.

**Pré-requisito:** `notebook_00_EDA.ipynb` | **Próximo passo:** `notebook_02_modeling.ipynb`

Todas as transformações aplicadas ao conjunto de treino são aprendidas **exclusivamente a partir do treino** e apenas aplicadas (sem re-ajuste) ao conjunto de teste, garantindo ausência de data leakage em qualquer etapa da pipeline.

| Componente | Baseline | Otimizado | Justificativa |
|---|---|---|---|
| TF-IDF `max_features` | 5.000 | 12.000 | Vocabulário mais rico captura termos compostos |
| `ngram_range` | (1,2) | (1,3) | Trigramas capturam padrões diagnósticos |
| `min_df` / `max_df` | - | 2 / 0.95 | Remove hapax e stopwords remanescentes |
| Features de estilo | 3 | 15 | EDA revelou features adicionais discriminativas |
| Lematização | - | Sim (NLTK+POS) | Reduz variação morfológica preservando semântica |

## 1. Importações e Configurações

Além das bibliotecas padrão de análise e transformação, este notebook utiliza `scipy.sparse` para manipulação eficiente de matrizes esparsas - necessário dado que a matriz TF-IDF possui dimensionalidade de 12.000 colunas com alta esparsidade. A função `find_root()` detecta automaticamente a raiz do projeto, garantindo portabilidade entre ambientes (local, Kaggle, Colab).

In [1]:
# ── Dependências do notebook ──────────────────────────────────────────────
# scipy.sparse: manipulação de matrizes esparsas (TF-IDF + style features)
# hstack: concatenação horizontal de matrizes esparsas sem densificação
# save_npz: serialização eficiente de matrizes esparsas em disco
import pickle
import re
import time
import warnings
from pathlib import Path

import nltk
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix, hstack, save_npz
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MaxAbsScaler

warnings.filterwarnings('ignore')

# --- Detecta automaticamente a raiz do projeto ---
# Sobe diretorios ate encontrar a pasta 'inputs/' com os CSVs
def find_root() -> Path:
    candidate = Path('.').resolve()
    for _ in range(6):  # sobe ate 6 niveis
        if (candidate / 'inputs' / 'train.csv').exists():
            return candidate
        candidate = candidate.parent
    raise FileNotFoundError(
        'Nao foi possivel encontrar a pasta inputs/train.csv. '
        'Execute o notebook a partir da raiz do projeto LIGIA_FINAL.'
    )

ROOT_DIR = find_root()
DATA_DIR = ROOT_DIR / 'outputs' / 'artifacts'
DATA_DIR.mkdir(parents=True, exist_ok=True)

print(f'ROOT_DIR : {ROOT_DIR}')
print(f'DATA_DIR : {DATA_DIR}')
# ── Modulo compartilhado de pre-processamento ──────────────────────────────
# As funcoes clean_text(), add_style_features(), lemmatize_series() e as
# constantes STYLE_COLS, SENSATIONAL_TERMS, REUTERS_SUBJECTS estao centralizadas
# em src/preprocessing.py para evitar duplicacao com notebook_03_inference.ipynb
import sys
# ROOT_DIR ja foi calculado acima pelo find_root() — garante que src/ seja encontrado
# independentemente de onde o notebook e executado (notebooks/, raiz, etc.)
if str(ROOT_DIR) not in sys.path:
    sys.path.insert(0, str(ROOT_DIR))
try:
    from src.preprocessing import (
        clean_text, add_style_features, lemmatize_series,
        STYLE_COLS, SENSATIONAL_TERMS, REUTERS_SUBJECTS, _CLEAN_PATTERNS,
    )
    print("Modulo src/preprocessing.py importado com sucesso.")
except ImportError as e:
    print(f"[AVISO] src/preprocessing.py nao encontrado ({e}). Funcoes definidas localmente abaixo.")


ROOT_DIR : C:\Users\gui05\Pictures\LIGIA_FINAL
DATA_DIR : C:\Users\gui05\Pictures\LIGIA_FINAL\outputs\artifacts
Modulo src/preprocessing.py importado com sucesso.


## 2. Remoção de Data Leakage

A EDA identificou que determinadas marcações textuais funcionam como identificadores diretos da classe, tornando a predição trivial e metodologicamente inválida. Especificamente, a assinatura `(Reuters)` está presente em mais de 99% dos artigos das categorias `politicsNews` e `worldnews`, enquanto fontes como `Infowars` e `Breitbart` são exclusivas da classe *fake*. Um modelo treinado sem remover esses artefatos aprenderia a identificar o veículo publicador, não os padrões linguísticos da desinformação.

**Artefatos removidos:**
- **Tags de localização/agência:** `WASHINGTON (Reuters) -` (exclusivas de notícias reais)
- **Menções a agências:** `(Reuters)`, `(AP)`, `(AFP)`
- **Fontes fake conhecidas:** `Infowars`, `Breitbart`, etc.
- **URLs, mentions e artefatos de coleta**

> **Importante:** As features de estilo são calculadas **antes** da limpeza de leakage, preservando o estilo tipográfico original do autor (maiúsculas, pontuação, etc.).

In [2]:
# ── Padrões Regex para remoção de data leakage ────────────────────────────
# RE_LOCATION_AGENCY: remove datelines no formato "WASHINGTON (Reuters) —"
#   presente exclusivamente em notícias reais (agências jornalísticas)
# RE_AGENCY_TAG: remove menções inline "(Reuters)", "(AP)", "(AFP)"
# RE_FAKE_SOURCES: remove nomes de veículos exclusivamente associados a fake news
# RE_URL / RE_MENTION: remove artefatos de coleta sem valor linguístico
# RE_LEAKAGE_ARTIFACTS: padrões residuais (ex: "21WIRE", "READ MORE")
RE_LOCATION_AGENCY   = re.compile(
    r'^[A-Z][A-Z\s/,\.]+\s*\([A-Za-z\s]+\)\s*[-\u2013\u2014]?\s*', re.MULTILINE)
RE_AGENCY_TAG        = re.compile(r'\(\s*(?:Reuters|AP|AFP)\s*\)', re.IGNORECASE)
RE_FAKE_SOURCES      = re.compile(
    r'\b(?:21st\s*Century\s*Wire|YourNewsWire|Infowars|Breitbart|RT\.com|NaturalNews|BeforeItsNews)\b',
    re.IGNORECASE,
)
RE_URL               = re.compile(r'https?://\S+|www\.\S+', re.IGNORECASE)
RE_MENTION           = re.compile(r'@\w+')
RE_LEAKAGE_ARTIFACTS = re.compile(
    r'\b(video|image|featured|getty(\s+images)?|bit\.ly|pic\.twitter\.com|reuters)\b',
    re.IGNORECASE | re.MULTILINE,
)

_CLEAN_PATTERNS = (
    RE_LOCATION_AGENCY, RE_AGENCY_TAG, RE_LEAKAGE_ARTIFACTS,
    RE_FAKE_SOURCES, RE_URL, RE_MENTION,
)

def clean_text(text: str) -> str:
    for pattern in _CLEAN_PATTERNS:
        text = pattern.sub('', str(text))
    return re.sub(r'\s{2,}', ' ', text).strip()

test_cases = [
    '(Reuters) - The president announced today https://reuters.com/article ...',
    'WASHINGTON (Reuters) - Senate passed the bill.',
    'Infowars reports: shocking news!',
]
print('Validacao dos patterns:')
for tc in test_cases:
    print(f'  Antes : {tc}')
    print(f'  Depois: {clean_text(tc)}')
    print()

Validacao dos patterns:
  Antes : (Reuters) - The president announced today https://reuters.com/article ...
  Depois: - The president announced today ...

  Antes : WASHINGTON (Reuters) - Senate passed the bill.
  Depois: Senate passed the bill.

  Antes : Infowars reports: shocking news!
  Depois: reports: shocking news!



## 3. Features de Estilo (15 features)

Fundamentadas nos achados da EDA, estas features capturam dimensões linguísticas não representadas diretamente pelo conteúdo textual. A hipótese é que a desinformação apresenta assinaturas estilísticas mensuráveis - uso excessivo de maiúsculas, pontuação agressiva e vocabulário sensacionalista - que são **ortogonais** ao conteúdo semântico capturado pelo TF-IDF.

As 15 features são divididas em três grupos funcionais:
- **Volume textual:** `text_len`, `word_count`, `title_len`, `avg_word_len`
- **Pontuação e tipografia:** `caps_ratio`, `title_caps_ratio`, `exclamation_count`, `question_count`, `ellipsis_count`, `quote_count`, `all_caps_words`
- **Léxico e estilo:** `sentence_count`, `avg_sentence_len`, `unique_word_ratio`, `sensational_count`

> **Ordem crítica:** Estas features são extraídas do **texto bruto**, antes da remoção de leakage, para preservar o estilo tipográfico original do autor.

In [3]:
# ── Extração das 15 features de estilo ────────────────────────────────────
# Calculadas no texto BRUTO (antes da limpeza) — ver justificativa na Seção 3
# caps_ratio: proporção de caracteres alfabéticos em maiúsculas no corpo do texto
# unique_word_ratio: diversidade lexical = palavras únicas / total de palavras
# sensational_count: contagem de termos de alarme/urgência (lista curada)
SENSATIONAL_RE = re.compile(
    r'\b(shocking|unbelievable|amazing|incredible|must see|breaking|exclusive|urgent)\b',
    re.IGNORECASE,
)

STYLE_COLS = [
    'caps_ratio', 'exclamation_count', 'title_len',
    'text_len', 'word_count', 'avg_word_len',
    'sentence_count', 'avg_sentence_len',
    'question_count', 'quote_count', 'ellipsis_count',
    'all_caps_words', 'title_caps_ratio',
    'unique_word_ratio', 'sensational_count',
]

def _caps_ratio(text: str) -> float:
    alpha = [c for c in str(text) if c.isalpha()]
    return sum(c.isupper() for c in alpha) / len(alpha) if alpha else 0.0

def _avg_word_len(text: str) -> float:
    words = str(text).split()
    return float(np.mean([len(w) for w in words])) if words else 0.0

def _unique_word_ratio(text: str) -> float:
    words = str(text).split()
    return len(set(words)) / len(words) if words else 0.0

def _quote_count(text: str) -> int:
    return sum(1 for c in str(text) if c in ('"', "'", '\u2018', '\u2019', '\u201c', '\u201d'))

def add_style_features(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df['caps_ratio']        = df['text'].apply(_caps_ratio)
    df['exclamation_count'] = df['text'].str.count('!')
    df['title_len']         = df['title'].str.len()
    df['text_len']          = df['text'].str.len()
    df['word_count']        = df['text'].str.split().str.len()
    df['avg_word_len']      = df['text'].apply(_avg_word_len)
    df['sentence_count']    = df['text'].str.count(r'[.!?]+')
    df['avg_sentence_len']  = df['word_count'] / df['sentence_count'].replace(0, 1)
    df['question_count']    = df['text'].str.count(r'\?')
    df['quote_count']       = df['text'].apply(_quote_count)
    df['ellipsis_count']    = df['text'].str.count(r'\.{2,}')
    df['all_caps_words']    = df['text'].str.findall(r'\b[A-Z]{2,}\b').str.len()
    df['title_caps_ratio']  = df['title'].apply(_caps_ratio)
    df['unique_word_ratio'] = df['text'].apply(_unique_word_ratio)
    df['sensational_count'] = df['text'].apply(lambda x: len(SENSATIONAL_RE.findall(str(x))))
    return df

print(f'Features de estilo definidas: {len(STYLE_COLS)} colunas')
print(STYLE_COLS)

Features de estilo definidas: 15 colunas
['caps_ratio', 'exclamation_count', 'title_len', 'text_len', 'word_count', 'avg_word_len', 'sentence_count', 'avg_sentence_len', 'question_count', 'quote_count', 'ellipsis_count', 'all_caps_words', 'title_caps_ratio', 'unique_word_ratio', 'sensational_count']


## 4. Pipeline de Pré-processamento

A função `preprocess()` encapsula as quatro etapas de transformação em uma ordem determinística, projetada para eliminar qualquer possibilidade de contaminação entre treino e teste:

1. **Extração de features de estilo** no texto bruto - antes de qualquer limpeza
2. **Remoção de leakage** via expressões regulares
3. **Concatenação** de título limpo + texto limpo em `clean_text`
4. **Remoção de colunas originais** para evitar uso acidental nas etapas seguintes

A função recebe o parâmetro `split` (ex: `"TRAIN"`, `"TEST"`) exclusivamente para fins de logging - a lógica de transformação é idêntica para ambos os conjuntos.

In [4]:
_DROP = ['subject', 'date', 'title', 'text', 'title_clean', 'text_clean']

def preprocess(df: pd.DataFrame, split: str) -> pd.DataFrame:
    """
    Etapas em ordem determinística — a sequência é crítica para evitar leakage:
    1. Features de estilo NO TEXTO BRUTO (antes de qualquer limpeza)
    2. Limpeza de leakage
    3. Concatenacao titulo+texto limpos
    4. Remoção das colunas originais (title, text, date) — evita uso acidental na modelagem
    """
    print(f'[{split}] Shape inicial: {df.shape}')
    df = df.copy()
    df['title'] = df['title'].fillna('')
    df['text']  = df['text'].fillna('')
    df = add_style_features(df)
    df['title_clean'] = df['title'].apply(clean_text)
    df['text_clean']  = df['text'].apply(clean_text)
    df['clean_text']  = (df['title_clean'] + ' ' + df['text_clean']).str.strip()
    df = df.drop(columns=[c for c in _DROP if c in df.columns])
    print(f'[{split}] Shape final  : {df.shape}')
    return df

print('Pipeline definido.')

Pipeline definido.


## 5. Carregamento e Validação dos Dados

Antes de aplicar qualquer transformação, verifica-se a integridade estrutural dos dados: presença das colunas esperadas e ausência de sobreposição de IDs entre treino e teste. A validação de overlap é uma salvaguarda contra data leakage no nível de instâncias - se um exemplo de teste aparecesse no treino, o modelo teria vantagem injusta na avaliação.

In [5]:
# ── Carregamento e validação de integridade dos datasets ──────────────────
# O assert de overlap verifica que nenhum ID do teste está no treino
# (data leakage em nível de instância — invalidaria a avaliação competitiva)
train_path = ROOT_DIR / 'inputs' / 'train.csv'
test_path  = ROOT_DIR / 'inputs' / 'test.csv'

df_train_raw = pd.read_csv(train_path)
df_test_raw  = pd.read_csv(test_path)

print(f'Train : {df_train_raw.shape}  |  Test : {df_test_raw.shape}')
print(f'Colunas train: {list(df_train_raw.columns)}')

# Validacao: sem overlap de IDs
assert 'id' in df_train_raw.columns and 'id' in df_test_raw.columns
overlap = set(df_train_raw['id']) & set(df_test_raw['id'])
assert len(overlap) == 0, f'Data leakage: {len(overlap)} IDs comuns!'
print('Validacao de overlap: OK (0 IDs comuns)')

# Validacao: sem NaN na label
assert df_train_raw['label'].isna().sum() == 0
print('Validacao NaN em label: OK')

print(f'\nDistribuicao de classes (train):')
print(df_train_raw['label'].value_counts().rename({0: 'Real (0)', 1: 'Fake (1)'}))

Train : (22844, 6)  |  Test : (5712, 5)
Colunas train: ['id', 'title', 'text', 'subject', 'date', 'label']
Validacao de overlap: OK (0 IDs comuns)
Validacao NaN em label: OK

Distribuicao de classes (train):
label
Real (0)    17133
Fake (1)     5711
Name: count, dtype: int64


## 6. Aplicação do Pipeline

O pipeline é aplicado independentemente a treino e teste. O assert final verifica que todas as 15 features de estilo foram criadas nos dois conjuntos, garantindo que a matriz de features terá o mesmo número de colunas durante treino e inferência.

In [6]:
# ── Aplicação do pipeline de pré-processamento ────────────────────────────
# O pipeline é aplicado de forma idêntica a TRAIN e TEST
# O assert verifica consistência de colunas antes da vetorização
df_train = preprocess(df_train_raw, 'TRAIN')
df_test  = preprocess(df_test_raw,  'TEST')

assert all(c in df_test.columns for c in STYLE_COLS), 'Features de estilo faltando no test!'
print('\nIntegridade das features: OK')

[TRAIN] Shape inicial: (22844, 6)
[TRAIN] Shape final  : (22844, 18)
[TEST] Shape inicial: (5712, 5)
[TEST] Shape final  : (5712, 17)

Integridade das features: OK


## 7. Lematização com NLTK

A lematização com POS tagging reduz formas flexionadas à sua forma canônica: `running` → `run`, `better` → `good`, `policies` → `policy`. Diferentemente do stemming (que trunca sufixos mecanicamente), a lematização com `WordNetLemmatizer` utiliza o contexto morfossintático da palavra (verbo, substantivo, adjetivo) para produzir formas válidas no dicionário.

**Justificativa:** Reduz a dimensionalidade efetiva do vocabulário sem perda semântica, consolidando variantes morfológicas de um mesmo conceito em um único token. Isso diminui o overfitting do TF-IDF a formas específicas de palavras.

**Custo computacional:** A combinação de tokenização + POS tagging + lematização tem custo O(n) no número de tokens. Para o volume do dataset, o processo pode levar alguns minutos - o progresso é reportado via `tqdm`.

> **Recursos NLTK necessários:** `punkt_tab`, `wordnet`, `stopwords`, `averaged_perceptron_tagger_eng` - baixados automaticamente na célula abaixo.

In [7]:
# ── Lematização com POS tagging (NLTK) ────────────────────────────────────
# POS tagging (Part-of-Speech) melhora a qualidade da lematização ao fornecer
# o contexto gramatical de cada token ao WordNetLemmatizer
# _POS_MAP: converte tags Penn Treebank (JJ, VB, NN, RB) para tags WordNet
# Tokens com len < 2 e stopwords são removidos para reduzir ruído
for resource in ['punkt_tab', 'wordnet', 'stopwords', 'averaged_perceptron_tagger_eng']:
    nltk.download(resource, quiet=True)

from nltk import pos_tag
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

_lemmatizer = WordNetLemmatizer()
_stop_words = set(stopwords.words('english'))
_POS_MAP    = {'J': wordnet.ADJ, 'V': wordnet.VERB, 'N': wordnet.NOUN, 'R': wordnet.ADV}

def _wn_pos(tag: str) -> str:
    return _POS_MAP.get(tag[0], wordnet.NOUN)

def lemmatize_series(texts: pd.Series, split: str = '') -> pd.Series:
    t0     = time.time()
    result = []
    for text in texts:
        tokens = word_tokenize(str(text).lower())
        lemmas = [
            _lemmatizer.lemmatize(w, _wn_pos(t))
            for w, t in pos_tag(tokens)
            if w.isalpha() and len(w) > 1 and w not in _stop_words
        ]
        result.append(' '.join(lemmas))
    elapsed = time.time() - t0
    print(f'  [{split}] Lematizacao: {len(texts):,} textos em {elapsed:.1f}s')
    return pd.Series(result, index=texts.index)

print('Lematizando TRAIN...')
df_train['clean_text'] = lemmatize_series(df_train['clean_text'], 'TRAIN')
print('Lematizando TEST...')
df_test['clean_text']  = lemmatize_series(df_test['clean_text'], 'TEST')

Lematizando TRAIN...
  [TRAIN] Lematizacao: 22,844 textos em 146.2s
Lematizando TEST...
  [TEST] Lematizacao: 5,712 textos em 35.8s


## 8. Vetorização TF-IDF Otimizada

O TF-IDF (Term Frequency–Inverse Document Frequency) representa cada documento como um vetor esparso no espaço de termos, ponderando a relevância de cada termo pela sua frequência no documento e pela sua raridade no corpus. Os parâmetros abaixo foram definidos com base nos achados da EDA e em experimentos de ablação realizados durante o desenvolvimento.

| Parâmetro | Valor | Motivação |
|---|---|---|
| `max_features` | 12.000 | Vocabulário mais rico que o baseline (5.000) |
| `ngram_range` | (1,3) | Trigramas capturam padrões compostos diagnósticos |
| `min_df` | 2 | Remove hapax legomena (ruído estatístico) |
| `max_df` | 0.95 | Remove stopwords remanescentes |
| `sublinear_tf` | True | log(1+TF) reduz o efeito de termos muito frequentes |

> O TF-IDF é **fitado exclusivamente no TRAIN** e apenas transformado no TEST, prevenindo data leakage de vocabulário 

In [8]:
# ── Vetorização TF-IDF ────────────────────────────────────────────────────
# fit_transform no TRAIN: aprende o vocabulário e transforma em uma operação
# transform no TEST: aplica o vocabulário aprendido sem modificá-lo (anti-leakage)
# Resultado: matrizes esparsas de shape (n_docs, 12_000)
X_train_text = df_train['clean_text']
y_train      = df_train['label']
X_test_text  = df_test['clean_text']

tfidf = TfidfVectorizer(
    stop_words='english',
    max_features=12_000,
    min_df=2,
    max_df=0.95,
    sublinear_tf=True,
    ngram_range=(1, 3),
)

# FIT apenas no TRAIN - transform separado para evitar leakage
X_train_tfidf = tfidf.fit_transform(X_train_text)
X_test_tfidf  = tfidf.transform(X_test_text)

print(f'TF-IDF - Train: {X_train_tfidf.shape}  |  Test: {X_test_tfidf.shape}')
print(f'Vocabulario efetivo: {len(tfidf.vocabulary_):,} termos')

TF-IDF - Train: (22844, 12000)  |  Test: (5712, 12000)
Vocabulario efetivo: 12,000 termos


## 9. Combinação Final: TF-IDF + Style Features (scaled)

A matriz final de features combina dois tipos de representação complementares:

| Componente | Tipo | Notas |
|---|---|---|
| TF-IDF | Esparso | 12.000 termos, 1–3grams, sublinear_tf |
| Style features (scaled) | Denso convertido a esparso | 15 features, MaxAbsScaler |

**Por que `MaxAbsScaler` e não `StandardScaler`?**
O `StandardScaler` subtrai a média de cada feature, o que **densificaria** os zeros da matriz TF-IDF esparsa, resultando em uma matriz densa de ~12.000 colunas e potencial estouro de memória RAM. O `MaxAbsScaler` escala os valores para o intervalo [-1, 1] sem deslocar o centro, preservando a estrutura esparsa e permitindo a concatenação horizontal eficiente via `hstack`.

> **Decisão Técnica (Anti-Leakage):** Tanto o TF-IDF quanto o `MaxAbsScaler` são ajustados (`fit`) **exclusivamente no conjunto de TREINO** e aplicados (`transform`) no TEST, prevenindo leakage de escala e vocabulário.

In [10]:
# ── Escalonamento das features de estilo e concatenação final ───────────
# MaxAbsScaler: fit no TRAIN, transform no TEST (anti-leakage de escala)
# csr_matrix: converte o array denso de style features para formato esparso
#   → permite hstack com a matriz TF-IDF sem densificar nenhuma das duas
# hstack: concatenação horizontal eficiente em memória (scipy.sparse)
# --- Scaling das features de estilo (MaxAbsScaler) ---
# Fit apenas no TRAIN, transform no TEST — evita leakage de escala
style_scaler = MaxAbsScaler()
X_train_style_arr = df_train[STYLE_COLS].values.astype(float)
X_test_style_arr  = df_test[STYLE_COLS].values.astype(float)

X_train_style = csr_matrix(style_scaler.fit_transform(X_train_style_arr))
X_test_style  = csr_matrix(style_scaler.transform(X_test_style_arr))

# --- Combinacao final: TF-IDF + Style (scaled) ---
X_train_final = hstack([X_train_tfidf, X_train_style]).tocsr()
X_test_final  = hstack([X_test_tfidf,  X_test_style]).tocsr()

print('Composicao da matriz final:')
print(f'  TF-IDF features : {X_train_tfidf.shape[1]:,}')
print(f'  Style features  : {len(STYLE_COLS)} (scaled)')
print(f'  Total features  : {X_train_final.shape[1]:,}')
print(f'  X_train         : {X_train_final.shape}')
print(f'  X_test          : {X_test_final.shape}')

# Validacao anti-NaN
assert not np.isnan(X_train_style_arr).any(), 'NaN nas features de estilo (train)!'
assert not np.isnan(X_test_style_arr).any(),  'NaN nas features de estilo (test)!'
print('Validacao NaN/Inf: OK')

Composicao da matriz final:
  TF-IDF features : 12,000
  Style features  : 15 (scaled)
  Total features  : 12,015
  X_train         : (22844, 12015)
  X_test          : (5712, 12015)
Validacao NaN/Inf: OK


## 10. Salvamento dos Artefatos

Todos os artefatos gerados nesta etapa são serializados em disco para uso nas etapas seguintes (modelagem e inferência). É fundamental salvar não apenas as matrizes de features, mas também os transformadores ajustados (`tfidf_vectorizer.pkl`, `style_scaler.pkl`), pois eles serão necessários para processar novos documentos em inferência com o **mesmo vocabulário e escala** utilizados no treino.

In [11]:
# ── Serialização dos artefatos de pré-processamento ──────────────────────
# X_train.npz / X_test.npz: matrizes esparsas finais (TF-IDF + style, scaled)
# y_train.csv: rótulos de treino alinhados por índice com X_train
# tfidf_vectorizer.pkl: vocabulário e pesos IDF aprendidos no treino
# style_scaler.pkl: parâmetros de escala (MaxAbs) aprendidos no treino
# train/test_preprocessed.csv: DataFrames com features de estilo para análise
save_npz(DATA_DIR / 'X_train.npz', X_train_final)
save_npz(DATA_DIR / 'X_test.npz',  X_test_final)

y_train.to_csv(DATA_DIR / 'y_train.csv', index=False)
df_test[['id']].to_csv(DATA_DIR / 'test_ids.csv', index=False)

with open(DATA_DIR / 'tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(tfidf, f)

with open(DATA_DIR / 'style_scaler.pkl', 'wb') as f:
    pickle.dump(style_scaler, f)

df_train.to_csv(DATA_DIR / 'train_preprocessed.csv', index=False)
df_test.to_csv(DATA_DIR / 'test_preprocessed.csv', index=False)

print(f'Artefatos salvos em: {DATA_DIR}')
print(f'  X_train.npz            {X_train_final.shape}')
print(f'  X_test.npz             {X_test_final.shape}')
print(f'  y_train.csv')
print(f'  test_ids.csv')
print(f'  tfidf_vectorizer.pkl   {len(tfidf.vocabulary_):,} termos')
print(f'  style_scaler.pkl')
print(f'  train_preprocessed.csv')
print(f'  test_preprocessed.csv')
print(f'\nProximo passo: notebook_02_modeling.ipynb')

Artefatos salvos em: C:\Users\gui05\Pictures\LIGIA_FINAL\outputs\artifacts
  X_train.npz            (22844, 12015)
  X_test.npz             (5712, 12015)
  y_train.csv
  test_ids.csv
  tfidf_vectorizer.pkl   12,000 termos
  style_scaler.pkl
  train_preprocessed.csv
  test_preprocessed.csv

Proximo passo: notebook_02_modeling.ipynb
